In [1]:
from concurrent.futures import ThreadPoolExecutor
import time
import threading


class  CustomSum:
  def  __init__(self):
    self.value =  0

  def  update(self, n):
    print(n)
    tmp_value =  self.value
    tmp_value +=  1
    time.sleep(0.1)
    self.value = tmp_value


cs = CustomSum()

with ThreadPoolExecutor(max_workers=3) as executor:
  for n in ["first", "second", "third"]:
    executor.submit(cs.update, n)
print(cs.value)


first
second
third
1


In [3]:
from concurrent.futures import ThreadPoolExecutor
import time
import threading


class  CustomSum:
  def  __init__(self):
    self.value =  0
    self._lock = threading.Lock()

  def  update(self, n):
    print(n)
    self._lock.acquire()
    tmp_value =  self.value
    tmp_value +=  1
    time.sleep(0.1)
    self.value = tmp_value
    self._lock.release()


cs = CustomSum()

with ThreadPoolExecutor(max_workers=3) as executor:
  for n in ["first", "second", "third"]:
    executor.submit(cs.update, n)
print(cs.value)


first
second
third
3


In [7]:
from concurrent.futures import ThreadPoolExecutor
import time
import threading


class  CustomSum:
  def  __init__(self):
    self.value =  0
    self._lock = threading.Lock()

  def  update(self, n):
    print(n)
    with self._lock:
      tmp_value =  self.value
      tmp_value +=  1
      time.sleep(0.1)
      self.value = tmp_value


cs = CustomSum()
start_time = time.time()

with ThreadPoolExecutor(max_workers=3) as executor:
  for n in ["first", "second", "third"]:
    executor.submit(cs.update, n)
print(time.time()-start_time)

print(cs.value)


first
second
third
0.3031339645385742
3


In [6]:
import threading
import time

class  CustomSum:
  def  __init__(self):
    self.value =  0
    self._sem = threading.Semaphore(1)

    # self._lock = threading.Lock()

  def  update(self, n):
    print(n)
    with self._sem:
      tmp_value =  self.value
      tmp_value +=  1
      time.sleep(0.1)
      self.value = tmp_value


cs = CustomSum()

start_time = time.time()
with ThreadPoolExecutor(max_workers=3) as executor:
  for n in ["first", "second", "third"]:
    executor.submit(cs.update, n)
print(time.time()-start_time)
print(cs.value)


first
second
third
0.30513596534729004
3


In [5]:
import concurrent.futures
import queue
import random
import threading
import time


def send(queue, event):
    while not event.is_set():
        message = random.randint(1, 11)
        queue.put(message)
        print(f'send : {message}')
    print("send terminate")


def bucket(queue, event):
    while not event.is_set() or not queue.empty():
        try:
            message = queue.get(timeout=1)
            print(f'bucket : {message}')
        except queue.Empty:
            print("empty")
            pass
    print("bucket terminate")


pipeline = queue.Queue(maxsize=10)
event = threading.Event()

with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    executor.submit(send, pipeline, event)
    executor.submit(bucket, pipeline, event)
    time.sleep(0.1)
    event.set()


send : 2
send : 4
send : 9
send : 10
send : 3
send : 1
send : 8
send : 1
send : 7
send : 10
bucket : 2send : 2

bucket : 4
bucket : 9
bucket : 10
bucket : 3
bucket : 1
bucket : 8
bucket : 1
bucket : 7
bucket : 10
bucket : 2
send : 10bucket : 10

send : 7
send : 6
send : 4
send : 3
send : 11
send : 9
send : 11
send : 8
send : 11
send : 1
bucket : 7send : 8
bucket : 6
bucket : 4
send : 11
send : 10

bucket : 3
bucket : 11
bucket : 9
bucket : 11
bucket : 8
bucket : 11
bucket : 1
bucket : 8
bucket : 11
bucket : 10
send : 11
send : 2
send : 2
send : 1
send : 4
send : 6
send : 4
send : 5
send : 1
send : 11
bucket : 11
bucket : 2
bucket : 2
bucket : 1
bucket : 4
bucket : 6
bucket : 4
bucket : 5
bucket : 1
bucket : 11
send : 4
send : 11
send : 6
send : 1
send : 7
send : 6
send : 7
send : 4
send : 9
send : 7
bucket : 4
bucket : 11
bucket : 6
bucket : 1
bucket : 7
bucket : 6
bucket : 7
bucket : 4
bucket : 9
bucket : 7
send : 2
send : 8
send : 9
send : 11
send : 6
send : 6
send : 4
send : 5
send 

In [2]:
import concurrent.futures
import queue
import random
import threading
import time


def send(queue, event):
    while not event.is_set():
        message = random.randint(1, 11)
        queue.put(message)
        print(f'send : {message}')
    print("send terminate")


def bucket(queue, event):
    while not event.is_set() or not queue.empty():
        try:
            message = queue.get(timeout=1)
            print(f'bucket : {message}')
        except queue.Empty:
            print("empty")
            pass
    print("bucket terminate")


pipeline = queue.Queue(maxsize=10)
event = threading.Event()

with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    executor.submit(send, pipeline, event)
    executor.submit(bucket, pipeline, event)
    time.sleep(0.1)
    event.set()

send : 7
send : 10
send : 9
send : 5
send : 3
send : 5
send : 8
send : 3
send : 4
send : 3
bucket : 7send : 11

bucket : 10
bucket : 9
bucket : 5
bucket : 3
bucket : 5
bucket : 8send : 8
send : 5
send : 7
send : 6
send : 2
send : 4

bucket : 3
bucket : 4
send : 7
send : 8
bucket : 3send : 5

bucket : 11
bucket : 8
bucket : 5send : 2
send : 2
send : 2

bucket : 7
bucket : 6
bucket : 2
bucket : 4
bucket : 7
bucket : 8
bucket : 5
bucket : 2
bucket : 2
bucket : 2
send : 8
send : 10
send : 9
send : 4
send : 8
send : 8
send : 11
send : 10
send : 7
send : 10
bucket : 8send : 10

bucket : 10send : 11

bucket : 9send : 9

bucket : 4send : 6

bucket : 8send : 5

bucket : 8send : 1

bucket : 11send : 4

bucket : 10send : 7

bucket : 7send : 8

bucket : 10send : 7

bucket : 10send : 10

bucket : 11send : 8

bucket : 9send : 10

bucket : 6send : 5

bucket : 5send : 3

bucket : 1send : 11

bucket : 4send : 3

bucket : 7send : 3

bucket : 8send : 9

bucket : 7send : 9

bucket : 10send : 7

bucket : 8